### OLAP Queries

In [10]:
import psycopg2 as pg
import pandas as pd

HOST = "10.101.151.25"
DATABASE = "tpd017"
USER = "tpd017"
PASSWORD = "tpd017" 

# Execute a single statement.
def execute_single_sql_statement(sql):
    conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)    
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    
    
def getSQLfromQuery(sql, params):
    conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
    cur  = conn.cursor()    
    if len(params)==0:
        cur.execute(sql)
    else: 
        cur.execute(sql, params)
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    cur.close()
    conn.close()
    df=pandafy(data, colnames)
    return df

def pandafy(rows, colnames):
    N=len(colnames)
    D={cn: [] for cn in colnames}
    for row in rows:
        for i in range(N): D[colnames[i]].append(row[i])
    pdfy=pd.DataFrame(D)
    return pdfy


3. **Assumindo que os diferentes níveis de urbanismo nos municípios afetam a quantidade de tráfego nos mesmos, esse aumento também vai ocorrer no número de acidentes e na severidade dos mesmos?** 


* Análise e visualização, para cada nível de urbanismo, do número de acidentes, e da distribuição da severidade nos mesmo
* Dimensões relevantes para a query: Location, Severity
* Relevância para quais processos? Seguradoras, policiamento, estruturas de controlo de tráfego

``
SELECT COUNT(ft.accidentid)
from facts_table ft, location_dimension l
WHERE ft.locationkey = l.location_key
GROUP BY (ft.severitykey, l.urban_rural_classification)
``


In [29]:
SQL="""
SELECT ft.severitykey, l.urban_rural_classification, COUNT(ft.accidentid) AS number_of_accidents
from facts_table ft, location_dimension l
WHERE ft.locationkey = l.location_key
GROUP BY ft.severitykey, l.urban_rural_classification
ORDER BY ft.severitykey, l.urban_rural_classification
"""

df = getSQLfromQuery(SQL, ())

In [30]:
df

,severitykey,urban_rural_classification,number_of_accidents
0,1,Large Central Metro,5
1,1,Large Fringe Metro,4
2,1,Medium Metro,4
3,1,Non-Core,3
4,1,Small Metro,2
5,2,Large Central Metro,12664
6,2,Large Fringe Metro,4504
7,2,Medium Metro,5234
8,2,Micropolitan,2080
9,2,Non-Core,6178
